# QuAM Demonstration

In [1]:
import json
from quam.components import *
from quam.examples.superconducting_qubits import *

from pathlib import Path
root_folder = Path("./output")
root_folder.mkdir(exist_ok=True)
for key in ["basic", "referenced", "referenced_multifile"]:
    (root_folder / key).mkdir(exist_ok=True)

2024-05-04 06:30:43,277 - qm - INFO     - Starting session: 5fdcd5df-fd8e-4c5d-8708-16215135a260


## Creating QuAM from Scratch

The user defines his own QuAM by inheriting from `QuamRoot` (see `components/quam.py`).
Once defined, it can be instantiated as follows:

In [2]:
machine = QuAM()
machine

QuAM(qubits={}, wiring={})

We see that five attributes are defined in quam, all empty so far. We can begin populating it with different `QuamComponent`s

In [3]:
num_qubits = 2
for idx in range(num_qubits):
    # Create qubit components
    transmon = Transmon(
        id=idx,
        xy=IQChannel(
            opx_output_I=("con1", 3 * idx + 3),
            opx_output_Q=("con1", 3 * idx + 4),
            frequency_converter_up=FrequencyConverter(
                mixer=Mixer(),
                local_oscillator=LocalOscillator(power=10, frequency=6e9),
            ),
            intermediate_frequency=100e6,
        ),
        z=SingleChannel(opx_output=("con1", 3 * idx + 5)),
    )
    machine.qubits[transmon.name] = transmon
    readout_resonator = InOutIQChannel(
        id=idx,
        opx_output_I=("con1", 3 * idx + 1),
        opx_output_Q=("con1", 3 * idx + 2),
        opx_input_I=("con1", 1),
        opx_input_Q=("con1", 2,),
        frequency_converter_up=FrequencyConverter(
            mixer=Mixer(), local_oscillator=LocalOscillator(power=10, frequency=6e9)
        ),
    )
    transmon.resonator = readout_resonator

machine.print_summary()

QuAM:
  qubits: QuamDict
    q0: Transmon
      id: 0
      xy: IQChannel
        operations: QuamDict Empty
        id: None
        digital_outputs: QuamDict Empty
        opx_output_I: ('con1', 3)
        opx_output_Q: ('con1', 4)
        opx_output_offset_I: None
        opx_output_offset_Q: None
        frequency_converter_up: FrequencyConverter
          local_oscillator: LocalOscillator
            frequency: 6000000000.0
            power: 10
          mixer: Mixer
            local_oscillator_frequency: "#../local_oscillator/frequency"
            intermediate_frequency: "#../../intermediate_frequency"
            correction_gain: 0
            correction_phase: 0
          gain: None
        intermediate_frequency: 100000000.0
      z: SingleChannel
        operations: QuamDict Empty
        id: None
        digital_outputs: QuamDict Empty
        opx_output: ('con1', 5)
        filter_fir_taps: None
        filter_iir_taps: None
        opx_output_offset: None
        interm

A summary of QuAM can be shown using `machine.print_summary()`

## Saving and Loading QuAM

We can then save quam to a json file:

In [4]:
machine.save(root_folder / "basic" / "state.json")

Finally, we can also load quam from the same json file

In [5]:
loaded_quam = QuAM.load(root_folder / "basic" / "state.json")

Importantly, the `state.json` file only describes how QuAM is structured, and how the different QuAM components should be initialized. The components themselves are defined in their respective classes.

## Generating QUA Configuration

We can also generate the qua config from quam. This recursively calls `QuamComponent.apply_to_config()` on all quam components.

In [6]:
qua_config = machine.generate_config()
json.dump(qua_config, open(root_folder / "basic" / "qua_config.json", "w"), indent=2)

In [7]:
root_folder / "basic" / "qua_config.json"

PosixPath('output/basic/qua_config.json')

## QuAM Using References

QuAM allows values to reference another part of QuAM by setting its value to a string starting with a colon: `"#/path/to/referenced/value"`.

As an example why this is useful, we previously hardcoded all output ports. However, grouping everything at a top-level `"wiring"`` makes more sense, so we can have all output ports reference to it.

Below we also add several more references

In [8]:
num_qubits = 2
machine = QuAM()
machine.wiring = {
    "qubits": {
        f"q{idx}": {
            "port_I": ("con1", 3 * idx + 3),
            "port_Q": ("con1", 3 * idx + 4),
            "port_Z": ("con1", 3 * idx + 5),
        }
        for idx in range(num_qubits)
    },
    "feedline": {
        "opx_output_I": ("con1", 1),
        "opx_output_Q": ("con1", 2),
        "opx_input_I": ("con1", 1),
        "opx_input_Q": ("con1", 2),
    },
}

for idx in range(num_qubits):
    # Create qubit components
    transmon = Transmon(
        id=idx,
        xy=IQChannel(
            opx_output_I=f"#/wiring/qubits/q{idx}/port_I",
            opx_output_Q=f"#/wiring/qubits/q{idx}/port_Q",
            frequency_converter_up=FrequencyConverter(
                mixer=Mixer(),
                local_oscillator=LocalOscillator(power=10, frequency=6e9),
            ),
            intermediate_frequency=100e6,
        ),
        z=SingleChannel(opx_output=f"#/wiring/qubits/q{idx}/port_Z"),
    )
    machine.qubits[transmon.name] = transmon
    readout_resonator = InOutIQChannel(
        id=idx,
        opx_output_I="#/wiring/feedline/opx_output_I",
        opx_output_Q="#/wiring/feedline/opx_output_Q",
        opx_input_I="#/wiring/feedline/opx_input_I",
        opx_input_Q="#/wiring/feedline/opx_input_Q",
        frequency_converter_up=FrequencyConverter(
            mixer=Mixer(), local_oscillator=LocalOscillator(power=10, frequency=6e9)
        ),
    )
    transmon.resonator = readout_resonator

machine.print_summary()

QuAM:
  qubits: QuamDict
    q0: Transmon
      id: 0
      xy: IQChannel
        operations: QuamDict Empty
        id: None
        digital_outputs: QuamDict Empty
        opx_output_I: "#/wiring/qubits/q0/port_I"
        opx_output_Q: "#/wiring/qubits/q0/port_Q"
        opx_output_offset_I: None
        opx_output_offset_Q: None
        frequency_converter_up: FrequencyConverter
          local_oscillator: LocalOscillator
            frequency: 6000000000.0
            power: 10
          mixer: Mixer
            local_oscillator_frequency: "#../local_oscillator/frequency"
            intermediate_frequency: "#../../intermediate_frequency"
            correction_gain: 0
            correction_phase: 0
          gain: None
        intermediate_frequency: 100000000.0
      z: SingleChannel
        operations: QuamDict Empty
        id: None
        digital_outputs: QuamDict Empty
        opx_output: "#/wiring/qubits/q0/port_Z"
        filter_fir_taps: None
        filter_iir_taps: Non

We again save quam and the QUA config. The QUA config is identical to previous, but QuAM has changed significantly

In [9]:
machine.save(root_folder / "referenced" / "state.json")

In [10]:
qua_config = machine.generate_config()
json.dump(qua_config, open(root_folder / "referenced" / "qua_config.json", "w"), indent=2)

## Separating QuAM into Multiple Files

Finally, we can also separate parts of QuAM to be placed into a separate file.
This can be especially useful when combined with referencing because we can now have a separate file dedicated to the wiring of the experiment.

Here we point quam to a folder instead of a json file, and we specify that `"wiring"` should go to a separate `wiring.json` file in that folder:

In [11]:
machine.save(root_folder / "referenced_multifile" / "quam", content_mapping={"wiring.json": ["wiring"]})

It can subseuently be loaded as usual

In [12]:
machine = QuAM.load(root_folder / "referenced_multifile" / "quam")

Separating QuAM into multiple files allows the user to revert back to a previous QuAM but keep some parts the same.
For example, the user may want to revert all experimental settings, but the wiring of the setup should of course not change.